In [1]:
from openpyxl import *

wb_name = "CurvesFromBB\EUR.OIS.xlsx"
ws_name = 'Bcurve'
col_names = ['Term','InstType','InstDes','Mid','Frequency','DayCount']



In [2]:




def get_ref(col_names,ws):

    for row in ws.rows:
        retval ={}
        for cell in row:
            for name in col_names:
                if(cell.value == name):
                    retval[name] = cell
            
        if(len(retval) == len(col_names)):
            return retval
      
    raise Exception("Could not find columns:" + str(col_names ))
      

                
def get_list_from_cols(col_names,ws):
    retval = []

    refs = get_ref(col_names,ws)
    for index in range(1,500):     
        row = {}
        for col_name in col_names:
            row[col_name] = refs[col_name].offset(index,0).value
            if (row[col_name] is None):
                return retval            
        retval.append(row)
    return retval
    


In [3]:
wb = load_workbook(wb_name)
ws = wb.get_sheet_by_name(ws_name)

dict = get_list_from_cols(col_names,ws)
for item in dict:
        print(item)

{'InstType': u'CASH', 'Term': u'1 DY', 'Mid': -0.057, 'Frequency': u'ZERO', 'DayCount': u'ACT_360_EOMC', 'InstDes': u'Effective Overnight Index Aver'}
{'InstType': u'SWAP', 'Term': u'1 WK', 'Mid': -0.056, 'Frequency': u'ANNUAL', 'DayCount': u'ACT_360_EOMC', 'InstDes': u'EUR SWAP (EONIA)    1 WK'}
{'InstType': u'SWAP', 'Term': u'2 WK', 'Mid': -0.05575, 'Frequency': u'ANNUAL', 'DayCount': u'ACT_360_EOMC', 'InstDes': u'EUR SWAP (EONIA)    2 WK'}
{'InstType': u'SWAP', 'Term': u'1 MO', 'Mid': -0.063, 'Frequency': u'ANNUAL', 'DayCount': u'ACT_360_EOMC', 'InstDes': u'EUR SWAP (EONIA)    1 MO'}
{'InstType': u'SWAP', 'Term': u'2 MO', 'Mid': -0.073, 'Frequency': u'ANNUAL', 'DayCount': u'ACT_360_EOMC', 'InstDes': u'EUR SWAP (EONIA)    2 MO'}
{'InstType': u'SWAP', 'Term': u'3 MO', 'Mid': -0.081, 'Frequency': u'ANNUAL', 'DayCount': u'ACT_360_EOMC', 'InstDes': u'EUR SWAP (EONIA)    3 MO'}
{'InstType': u'SWAP', 'Term': u'4 MO', 'Mid': -0.087, 'Frequency': u'ANNUAL', 'DayCount': u'ACT_360_EOMC', 'Inst

C:\Users\fhoefer\AppData\Local\Continuum\Anaconda2\lib\site-packages\openpyxl\workbook\names\named_range.py:121: UserWarning: Discarded range with reserved name
  warnings.warn("Discarded range with reserved name")


In [21]:
import QuantLib as ql
#from ReadExcel import *
from openpyxl import *
import re

def printDatum(data,digits=2):
    format = '%%.%df %%%%' % digits
    return  "Maturity: " + str(ql.Period(data.numTimeUnits,data.timeUnit))+ "\tQuoted Rate:"+format % (100*data.rate)

def printOISwapDatum(data,digits=2):
    format = '%%.%df %%%%' % digits
    return  "Maturity: " + str(ql.Period(data.numTimeUnits,data.timeUnit))+ "\tQuoted Rate:"+format % (100*data.rate)

def printLIBORSwapDatum(data,digits=2):
    format = '%%.%df %%%%' % digits
    return  "Maturity: " + str(ql.Period(data.numTermUnits,data.termUnit))
    + "\t(Index:"  + str(ql.Period(data.numIndexUnits,data.indexUnit)) + ")"
    + "\tQuoted Rate:"+format % (100*data.rate)

def formatPrice(p,digits=2):
    format = '%%.%df' % digits
    return format % p

def formatRate(r,digits=2):
    format = '%%.%df %%%%' % digits
    return format % (r*100)

def formatVol(v, digits = 2):
    format = '%%.%df %%%%' % digits
    return format % (v * 100)

def formatPrice(p, digits = 2):
    format = '%%.%df' % digits
    return format % p

class MarketConventions(object):
    def __init__(self):
        self.settlementDays = 2
        self.calendar = ql.TARGET()                    
        self.fixedEoniaConvention = ql.ModifiedFollowing
        self.floatingEoniaConvention = ql.ModifiedFollowing
        self.fixedEoniaPeriod = 1 * ql.Years
        self.floatingEoniaPeriod = 1 * ql.Years
        self.fixedEoniaDayCount = ql.Actual360()
        self.floatingEoniaDayCount = ql.Actual360()        
        self.fixedSwapConvention = ql.ModifiedFollowing
        self.fixedSwapFrequency = ql.Annual
        self.fixedSwapDayCount = ql.Thirty360(ql.Thirty360.BondBasis)                          
        self.floatSwapConvention = ql.ModifiedFollowing
        self.floatSwapFrequency = ql.Semiannual
        self.floatSwapDayCount = ql.Actual360()



class Datum(object):
    """Whatever."""
    def __init__(self, numSettlementDays,timeUnit,numTimeUnits,rate):
        """Arguments:"""
        """numSettlementDays:\tNumber of days between inception and accrue start"""
        """timeUnit:\tTime unit ofthe accrue periods length"""
        """numTimeUnits:\tNumber of time units in the accrue period"""
        """rate:\tThe Coupon"""
        self.numSettlementDays = numSettlementDays
        self.timeUnit = timeUnit
        self.numTimeUnits = numTimeUnits
        self.rate = rate
    def __repr__(self):
        """Returns a string representation of this object that can
        be evaluated as a Python expression."""
        return printDatum(self,4)
    __str__ = __repr__
    """The str and repr forms of this object are the same."""

 
            
class SwapDatum(object):
    """Fra Quote along with instrument data."""
    def __init__(self,settlementDays,indexUnit,numIndexUnits,termUnit,numTermUnits,rate):
        self.settlementDays = settlementDays
        self.indexUnit = indexUnit
        self.numIndexUnits = numIndexUnits
        self.termUnit = termUnit
        self.numTermUnits = numTermUnits
        self.rate = rate
    
    def __repr__(self):
        """Returns a string representation of this object that can
        be evaluated as a Python expression."""
        return   printOISwapDatum(self,4)
    
    __str__ = __repr__
    """The str and repr forms of this object are the same."""


class MarketData(object):
     def __init__(self,depoQuotes,oiSwapQuotes,swapQuotes):
        self.depoQuotes = depoQuotes
        self.swapQuotes = swapQuotes
        self.oiSwapQuotes = oiSwapQuotes

def map_BBtenor_to_quantlib(tenor):
    if tenor == "YR":
        return ql.Years
    elif tenor == "MO":
        return ql.Months
    elif tenor == "WK":
        return ql.Weeks
    elif tenor == "DY":
        return ql.Days
    else:
        raise Exception("Faild to map tenor:" + str(tenor))

def getMarketData(path, ws_name, col_names):
    wb = load_workbook(path)
    ws = wb.get_sheet_by_name(ws_name)
    list = get_list_from_cols(col_names,ws)    

    
    depoQuotes = []
    oiSwapQuotes = []
    SwapQuotes = []

    for item in list:
        if ( ('InstType' in item) & (item['InstType'] == 'CASH')):
            numSettlementDays =0
            matchResult = re.match("([0-9]{1,2}) ([a-zA-Z][a-zA-Z])", item['Term'], flags=0)
            if(matchResult):
                timeUnit = map_BBtenor_to_quantlib(matchResult.group(2))
                numTimeUnits = int(matchResult.group(1))
                rate = item['Mid']
                depoQuotes.append(Datum(numSettlementDays,timeUnit,numTimeUnits,rate/100))
            else:
                print("Could not parse " + item['InstType'] + ' ' + 'Term' + ": " +item['Term'] )
        elif(('InstType' in item) & (item['InstType'] == 'SWAP' )
             & ('InstDes' in item)  & ( 'EONIA' in item['InstDes'])):
            numSettlementDays =0
            matchResult = re.match("([0-9]{1,2}) ([a-zA-Z][a-zA-Z])", item['Term'], flags=0)
            if(matchResult):
                timeUnit = map_BBtenor_to_quantlib(matchResult.group(2))
                numTimeUnits = int(matchResult.group(1))
                rate = item['Mid']
                oiSwapQuotes.append(Datum(numSettlementDays,timeUnit,numTimeUnits,rate/100))        
        else:
            print("Could not parse " + item['InstType'] + ' ' + 'Term' + ": " +item['Term'] )

    return MarketData(depoQuotes,oiSwapQuotes,SwapQuotes)





In [23]:
mdata = getMarketData("CurvesFromBB\EUR.OIS.xlsx",'Bcurve',['Term','InstType','InstDes','Mid','Frequency','DayCount'])
print(len(mdata.depoQuotes))
for item in mdata.depoQuotes:
    print(item)
for item in mdata.oiSwapQuotes:
    print(item)

Could not parse FUTURE Term: 10F
Could not parse FUTURE Term: 11F
Could not parse FUTURE Term: 12F
Could not parse FUTURE Term: 13F
Could not parse FUTURE Term: 14F
Could not parse FUTURE Term: 15F
Could not parse FUTURE Term: 16F
Could not parse FUTURE Term: 17F
Could not parse FUTURE Term: 18F
Could not parse FUTURE Term: 19F
Could not parse FUTURE Term: 20F
Could not parse FUTURE Term: 21F
Could not parse FUTURE Term: 22F
Could not parse FUTURE Term: 23F
Could not parse FUTURE Term: 24F
Could not parse SERIAL_FUTURE Term: 1SF
Could not parse SERIAL_FUTURE Term: 2SF
Could not parse SERIAL_FUTURE Term: 3SF
Could not parse SERIAL_FUTURE Term: 4SF
Could not parse SERIAL_FUTURE Term: 5SF
Could not parse SERIAL_FUTURE Term: 6SF
Could not parse SERIAL_FUTURE Term: 7SF
Could not parse SERIAL_FUTURE Term: 8SF
Could not parse SERIAL_FUTURE Term: 9SF
Could not parse SERIAL_FUTURE Term: 10SF
Could not parse SERIAL_FUTURE Term: 11SF
Could not parse SERIAL_FUTURE Term: 12SF
Could not parse SERIAL